# 데이터 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/My Drive/greenDevil/hospital73_addyearoffset.csv'

import pandas as pd
df = pd.read_csv(file_path)

In [ ]:
df.shape

(31830, 37)

# 병원 입원 순서 추정이 불가능한 열들에 대하여: 하나의 열만 랜덤하게 선택

관련 columns:
*   uniquepid
*   ~~patienthealthsystemstayid~~
*   age
*   hospitaldischargeyear
*   unitdischargestatus
*   hospitaldischargestatus
*   unitvisitnumber
*   hospitaladmityear  




1. Select columns

In [ ]:
group_cols = ['uniquepid', 'age','hospitaldischargeyear','unitdischargestatus','hospitaldischargestatus','unitvisitnumber','hospitaladmityear']
#group_cols = ['uniquepid', 'age','hospitaldischargeyear','unitdischargestatus','hospitaldischargestatus','hospitaladmityear']

  2. Function to identify distinct groups based on group_cols

In [ ]:
def select_one_group(g):
    selected_id = g['patienthealthsystemstayid'].sample(1).iloc[0]
    return g[g['patienthealthsystemstayid'] == selected_id]

3. Within each group, randomly select one 'patienthealthsystemstayid' group to keep

In [ ]:
df_dedup = df.groupby(group_cols, group_keys=False).apply(select_one_group)

/tmp/ipython-input-64-2157855530.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_dedup = df.groupby(group_cols, group_keys=False).apply(select_one_group)


In [ ]:
# Check dimensions
row_before = df.shape[0]
row_after = df_dedup.shape[0]
row_reduction_pct = ((row_before - row_after) / row_before) * 100
print("Original:", df.shape)
print("Processed:", df_dedup.shape)
print(f"Row reduction: {row_reduction_pct:.2f}%")

Original: (31830, 37)
Processed: (30323, 37)
Row reduction: 4.73%


4. 퇴원 상태 관련 처리

In [ ]:
# 1. 그룹 기준 정의
group_cols_full = ['uniquepid', 'age', 'hospitaladmityear', 'hospitaldischargeyear',
                   'unitvisitnumber', 'unitdischargestatus', 'hospitaldischargestatus']

# 그룹 내 모든 행을 유지해야 하므로, 먼저 각 그룹에 ID를 부여
df_dedup['group_id'] = df_dedup.groupby(group_cols_full).ngroup()

# 2. 각 그룹에 대해 대표 퇴원 상태 판단
def is_expired_group(g):
    return ((g['unitdischargestatus'] == 'Expired') |
            (g['hospitaldischargestatus'] == 'Expired')).any()

group_metadata = df_dedup.groupby('group_id').first().reset_index()
group_metadata['is_expired'] = df_dedup.groupby('group_id').apply(is_expired_group).values

# 3. 환자 기준(uniquepid + 조건 일부)으로 퇴원 상태가 다른 그룹들을 모음
# 퇴원 상태만 다르고 나머지는 같은 그룹을 비교하기 위한 기준
base_group_cols = ['uniquepid', 'age', 'hospitaladmityear', 'hospitaldischargeyear', 'unitvisitnumber']
group_metadata['join_key'] = group_metadata[base_group_cols].astype(str).agg('|'.join, axis=1)

# 4. 각 join_key 그룹에서 expired 그룹은 유지, 그 외는 하나 랜덤 선택
def select_groups(g):
    expired = g[g['is_expired']]
    non_expired = g[~g['is_expired']]
    selected = []
    if not expired.empty:
        selected.append(expired)
    if not non_expired.empty:
        selected.append(non_expired.sample(1))
    return pd.concat(selected)

selected_groups = group_metadata.groupby('join_key', group_keys=False).apply(select_groups)

# 5. 최종적으로 선택된 group_id만 추출하여 원래 데이터에서 필터링
final_group_ids = selected_groups['group_id'].unique()
df_final = df_dedup[df_dedup['group_id'].isin(final_group_ids)].drop(columns='group_id')

/tmp/ipython-input-67-3400820546.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  group_metadata['is_expired'] = df_dedup.groupby('group_id').apply(is_expired_group).values
/tmp/ipython-input-67-3400820546.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  selected_groups = group_metadata.groupby('join_key', group_keys=False).apply(select_groups)


In [ ]:
# Check dimensions
row_before = df.shape[0]
row_after = df_final.shape[0]
row_reduction_pct = ((row_before - row_after) / row_before) * 100
print("Original:", df.shape)
print("Processed (Final):", df_final.shape)
print(f"Row reduction: {row_reduction_pct:.2f}%")

Original: (31830, 37)
Processed (Final): (30323, 37)
Row reduction: 4.73%


In [ ]:
# Save
df_final.to_csv('/content/drive/My Drive/greenDevil/hospital73_SelectRandomStay.csv', index=False)